In [1]:
import pymongo
from tqdm import tqdm
import os
import markdown
import re
from nltk.stem import PorterStemmer
st=PorterStemmer()

/home/li/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
client = pymongo.MongoClient(host="192.168.0.106", port=29197)
client["github"].authenticate("github", "git332", "github")
db = client["github"]

In [16]:
db1 = db.npm_require_map
db2 = db.npm_require_map_detail
db3 = db.npm_tag_stem_top100

In [11]:
tag_map = {}
i = 0
for d in tqdm(db1.find()):
    i += 1
    if db2.find_one({"name":d["require"]}):
        d2 = db2.find_one({"name":d["require"]})
        if "keywords" in d2:
            for k in d2["keywords"]:
                if k.find("javascript") == -1 and k.find("js") == -1 and k.find("es") == -1 and k.find("lint") == -1 and k.find("node"):
                    k = st.stem(k)
                    if k in tag_map:
                        tag_map[k] += d["count"]
                    else:
                        tag_map[k] = d["count"]
                        
# for d in db2.find():
#     i += 1
#     if "keywords" in d:
#         for k in d["keywords"]:
#             if k in tag_map:
#                 tag_map[k] += 1
#             else:
#                 tag_map[k] = 1
                
# for d in db3.find():
#     i += 1
#     if "keywords" in d:
#         for k in d["keywords"]:
#             if k in tag_map:
#                 tag_map[k] += 1
#             else:
#                 tag_map[k] = 1

31930it [11:33, 46.01it/s]


In [6]:
len(tag_map)

22996

In [12]:
len(tag_map)

19663

In [14]:
sorted(tag_map.items(), key=lambda d: d[1], reverse=True)[:100]

[('babel', 40926),
 ('ast', 35500),
 ('compil', 34842),
 ('react', 30211),
 ('webpack', 27551),
 ('flavortown', 26769),
 ('transpil', 19134),
 ('browser', 18457),
 ('css', 18342),
 ('tdd', 15515),
 ('mocha', 15162),
 ('cli', 15054),
 ('assert', 15027),
 ('ecmascript', 13407),
 ('gruntplugin', 13121),
 ('loader', 12612),
 ('gulpplugin', 11954),
 ('bdd', 11614),
 ('util', 11008),
 ('plugin', 10032),
 ('webpack-load', 9910),
 ('babel-plugin', 9759),
 ('stream', 9712),
 ('librari', 9475),
 ('html', 9289),
 ('tap', 9268),
 ('ui', 9185),
 ('minifi', 9088),
 ('frontend', 9066),
 ('parser', 8995),
 ('server', 8512),
 ('static-code-analysi', 8255),
 ('web', 8164),
 ('file', 8121),
 ('http', 8012),
 ('karma-plugin', 7899),
 ('log', 7694),
 ('style', 7661),
 ('6to5', 7632),
 ('coverag', 7596),
 ('declar', 7497),
 ('qunit', 7250),
 ('framework', 7159),
 ('async', 7139),
 ('middlewar', 7101),
 ('modul', 7075),
 ('expect', 6899),
 ('vue', 6734),
 ('npm', 6731),
 ('lodash', 6488),
 ('jasmin', 6313),


In [15]:
composer_tag_top100 = sorted(tag_map.items(), key=lambda d: d[1], reverse=True)[:100]

In [17]:
for t in composer_tag_top100:
    db3.insert_one({"name":t[0],"count":t[1]})